# Bout, Stim, and Duration Post Hoc Analysis(CSV for Database) 

In [1]:
import joblib
import os
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
import numpy as np
from datetime import datetime
import matplotlib.pyplot as plt
import pickle
import time
from tkinter import *
import tkinter as tk
from tkinter import filedialog
from tqdm import tqdm
from tqdm import notebook
import itertools
import math
import seaborn as sns
import statistics 
from collections import OrderedDict
import csv

In [2]:
# Create Tk root
root = Tk()
# Hide the main window
root.withdraw()
root.call('wm', 'attributes', '.', '-topmost', True)
infiles = filedialog.askopenfilename(multiple=True, title='load posthoc-predictions.npy')

%gui tk

infiles

('Z:/KayCei/HDC/AH1-KO/120423-LIGHTS OFF/predict_video_exp_16.npy',)

In [3]:
# Check if the user selected any files
if infiles:
    # Assuming infiles is a list of selected file paths, let's use the first one
    selected_file = infiles[0]

    # Extract the directory path from the selected file's full path
    directory_path = os.path.dirname(selected_file)
    
#print(directory_path)

In [5]:
Random_List = [] #creates a array known as random list 
for f in range(len(infiles)):
    arr = np.load(infiles[f])
    Random_List.extend(arr) 
#print(Random_List)

In [6]:
behavior_labels = {
    0.0: "insignificant",
    1.0: "insignificant",
    2.0: "insignificant",
    3.0: "insignificant",
    4.0: "Rear",
    5.0: "insignificant",
    6.0: "insignificant",
    7.0: "Investigate Type 1",
    8.0: "Investigate Type 2", 
    9.0: "Contra-Itch",
    10.0: "Investigate Type 3",
    11.0: "insignificant",
    12.0: "insignificant",
    13.0: "insignificant",
    14.0: "Contra-Body Groom",
    15.0: "Face Groom Type 1",
    16.0: "Dive/Scrunch",
    17.0: "Head Groom",
    18.0: "Ipsi-Orient",
    19.0: "insignificant",
    20.0: "Face Groom Type 2",
    21.0: "Ipsi-Body Groom",
    22.0: "Ipsi-Itch Type 1",
    23.0: "insignificant",
    24.0: "insignificant",
    25.0: "Paw-Groom",
    26.0: "Locomotion",
    27.0: "insignificant",
    28.0: "Contra-Orient",
}

In [7]:
from collections import Counter

# Assuming Random_List contains behavior labels as values
label_counts = Counter(Random_List)

# Print the length of each behavior label
for label, count in label_counts.items():
    print(f"Label: {label}, Count: {count}")


Label: 26, Count: 13509
Label: 28, Count: 2211
Label: 18, Count: 2810
Label: 8, Count: 12643
Label: 27, Count: 678
Label: 3, Count: 940
Label: 15, Count: 342
Label: 7, Count: 1215
Label: 5, Count: 841
Label: 6, Count: 621
Label: 9, Count: 1136
Label: 16, Count: 341
Label: 22, Count: 237
Label: 23, Count: 303
Label: 10, Count: 591
Label: 21, Count: 203
Label: 25, Count: 23
Label: 24, Count: 64
Label: 17, Count: 74
Label: 14, Count: 224
Label: 4, Count: 55
Label: 1, Count: 54
Label: 19, Count: 116
Label: 11, Count: 30
Label: 13, Count: 33
Label: 20, Count: 4
Label: 12, Count: 19
Label: 0, Count: 13
Label: 2, Count: 1


# Total Time of Video in deciseconds

In [8]:
duration_in_deciseconds = len(Random_List)
duration_in_minutes = duration_in_deciseconds / 600
duration_in_frames = duration_in_deciseconds * 3

print('This video is', duration_in_deciseconds, 'deciseconds, which is', duration_in_minutes, 'minutes, which is', duration_in_frames, 'frames')
#print(Random_List)

This video is 39331 deciseconds, which is 65.55166666666666 minutes, which is 117993 frames


## Creating Array

## Time Integration (deciseconds) 

In [ ]:
BehaviorTime = []

for index, value in enumerate(Random_List):
    BehaviorTime.append((value, index))

In [ ]:
#print(BehaviorTime)

### Ensures you only analyze expected length of  Session 


length_of_session = (int(input("How long was the session in minutes?"))) * 600
print(length_of_session)
BehaviorTime = BehaviorTime[:(length_of_session+1)]

#### Removes all info after selected end time 

In [ ]:
print('This video is ', len(BehaviorTime), ' deciseconds, which is', (len(BehaviorTime)/600), 'minutes' )
print()
#print(BehaviorTime)

## Adds Stim Count and Frame Rate to Array 

In [ ]:
for z in BehaviorTime:
    behavior = z[0]
    time= z[1]
behavior_type_and_duration = []
current_behavior = BehaviorTime[0][0]
current_time = BehaviorTime[0][1]  
duration = 0
stim_count=0
frames = 0

for behavior, time in BehaviorTime:
    if behavior == current_behavior:
        start_time= current_time
        duration += 1
            
    else:
        if duration==1:
            end_time=start_time+1
        else:
            end_time= start_time + duration
        frames= duration*6
        stim_count= round(duration/6)
        behavior_type_and_duration.append((current_behavior, duration,frames,stim_count,start_time, end_time))
        current_behavior = behavior
        current_time = time
        start_time= time
        duration = 1
        end_time= time+1
        
# Append the final (current_behavior, current_time, count) tuple outside the loop
behavior_type_and_duration.append((current_behavior, duration, frames, stim_count, start_time, end_time))

# Make a copy of the original behavior_type_and_duration list
behaviortdt= list(behavior_type_and_duration)

In [ ]:
#print(behaviortdt)
#print(len(behaviortdt))

# Check Test Behavior Counts CSV

In [ ]:
# Initialize a dictionary to store total durations for each behavior label
total_durations = {label: 0 for label in behavior_labels.values()}

# Iterate through the behavior_type_and_duration list
for behavior, duration, _, _, _, _ in behavior_type_and_duration:
    # Get the behavior label based on the behavior code
    behavior_label = behavior_labels.get(behavior, "Unknown")
    
    # Add the duration to the total for the corresponding behavior label
    total_durations[behavior_label] += duration

# Print the total duration for each behavior label
for behavior_label, total_duration in total_durations.items():
    print(f"{behavior_label}: {total_duration}")



In [ ]:
#length_of_session = (int(input("How long was the session in minutes?"))) * 600 
#print(length_of_session) 
behaviortdt = [tup for tup in behaviortdt if tup[5] <= 39001]

In [ ]:
print('This video is ', behaviortdt[-1][-1], ' deciseconds, which is', (behaviortdt[-1][-1]/600), 'minutes' )
print()
#print(behaviortdt)

## Add Bout Count to the Array 

In [ ]:
enumerated_behaviortdt = []
behavior_index = {}  # Dictionary to store behavior and its corresponding index

for behavior, duration, frames, stim_count, start_time, end_time in behaviortdt:
    if behavior not in behavior_index:
        behavior_index[behavior] = 1
    else:
        behavior_index[behavior] += 1
    
    bout = behavior_index[behavior]
    enumerated_behaviortdt.append((behavior,stim_count,frames, bout, duration,start_time,end_time))

print(enumerated_behaviortdt)

# Stim Count Calcultaion

In [ ]:
#BCBDT = []

# Calculate durations and update BCBDT
#for behavior, duration, frames, bout, start_time, end_time in enumerated_behaviortdt:
    #stim_count = round(duration / 3)
    #if stim_count < 1:
        #stim_count = 0
    #if stim_count > 0:
        #BCBDT.append((behavior, duration, frames, bout, stim_count, start_time, end_time))

# Print the modified BCBDT
#print("Modified BCBDT:", BCBDT)

# Assign enumerated_behaviortdt to BCBDST
BCBDST = enumerated_behaviortdt


In [ ]:
behavior_labels = {
    0.0: "insignificant",
    1.0: "insignificant",
    2.0: "insignificant",
    3.0: "insignificant",
    4.0: "Rear",
    5.0: "insignificant",
    6.0: "insignificant",
    7.0: "Investigate Type 1",
    8.0: "Investigate Type 2", 
    9.0: "Contra-Itch",
    10.0: "Investigate Type 3",
    11.0: "insignificant",
    12.0: "insignificant",
    13.0: "insignificant",
    14.0: "Contra-Body Groom",
    15.0: "Face Groom Type 1", 
    16.0: "Dive/Scrunch",
    17.0: "Head Groom",
    18.0: "Ipsi-Orient",
    19.0: "insignificant",
    20.0: "Face Groom Type 2",
    21.0: "Ipsi-Body Groom",
    22.0: "Ipsi-Itch Type 1",
    23.0: "insignificant",
    24.0: "insignificant",
    25.0: "Paw-Groom",
    26.0: "Locomotion",
    27.0: "insignificant",
    28.0: "Contra-Orient",
}

bdbt_dict = {}  # Dictionary to store lists of behaviors

for behavior,stim_count,frames, bout, duration,start_time,end_time in BCBDST:
    behavior_label = behavior_labels.get(behavior, "Unknown Behavior")
    if behavior_label not in bdbt_dict:
        bdbt_dict[behavior_label] = []
    bdbt_dict[behavior_label].append((behavior, stim_count, frames, bout, duration, start_time,end_time))


In [ ]:


#print(bdbt_dict)

In [ ]:
keys_list = bdbt_dict.keys()
print(keys_list)

## Behavior of Interest

In [ ]:
for file_path in infiles: # if taking from Z Drive 
    folders = (os.path.dirname(file_path)).split('/')
    print(folders)
    print(len(folders))
    # Check if there are at least 5 elements in the 'folders' list
    if len(folders) >= 5:
        fol3 = folders[3].split('-')  # Split element at position 3 by hyphen
        fol4 = folders[4].split('-')  # Split element at position 4 by hyphen
        
        print("Folder 3:", fol3)
        print("Folder 4:", fol4)
    else:
        print("The folder path does not have enough elements to access positions 3 and 4.")
    

## Stim Count and Average Duration 

In [ ]:
from tabulate import tabulate
import matplotlib.pyplot as plt
import csv


def round_float(value):
    return round(value, 3)

average_duration_by_key = {}

for key, tuples_list in bdbt_dict.items():
    #if key=='insignificant':
        #continue #skip "insignificant" key and its data
    durations = [tuple_item[4] for tuple_item in tuples_list]
    average_duration = (sum(durations) / len(durations))/10
    average_duration_by_key[key] =average_duration

table_data = [(key, avg_duration) for key, avg_duration in average_duration_by_key.items()]
headers = ["Key", "Average Duration(seconds)"]

table = tabulate(table_data, headers, tablefmt="pretty")
print(table)

# Create a dictionary for the CSV data
csv_data = {f'{fol3[0]} {fol4[1]} Duration': average_duration_by_key}

# Plot the data
plt.figure(figsize=(10, 6))
plt.bar(average_duration_by_key.keys(), average_duration_by_key.values())
plt.xlabel("Key")
plt.ylabel("Average Duration (seconds)")
plt.title(f"Average Durations by Key {fol3[0]} {fol4[1]}")
plt.xticks(rotation=45, ha='right')
plt.tight_layout()

## Check if the user selected any files
if infiles:
    # Assuming infiles is a list of selected file paths, let's use the first one
    selected_file = infiles[0]

    # Extract the directory path from the selected file's full path
    directory_path = os.path.dirname(selected_file)

    # Save the plot in the same directory as the selected file with the specified filename
    filename = f'{fol3[0]} {fol4[1]} Duration.png'
    save_path = os.path.join(directory_path, filename)
    plt.savefig(save_path)
    print(f"Plot saved in: {save_path}")

    # Save the CSV data to a CSV file
    csv_filename = f'{fol3[0]} {fol4[1]} Duration.csv'
        # Construct the full path for saving the CSV in the same directory
    csv_save_path = os.path.join(directory_path, csv_filename)

    # Save the CSV data to the CSV file
    with open(csv_save_path, mode="w", newline="") as csv_file:
        csv_writer = csv.writer(csv_file)
        for dictionary_name, data_dict in csv_data.items():
            csv_writer.writerow([dictionary_name])  # Write the dictionary name as a header
            csv_writer.writerow(["Key", "Average Duration (seconds)"])
            for key, avg_duration in data_dict.items():
                csv_writer.writerow([key, avg_duration])

    print(f"CSV data saved in: {csv_save_path}")
    

In [ ]:
from tabulate import tabulate
import matplotlib.pyplot as plt

In [ ]:
total_count_by_key = {}

for key, tuples_list in bdbt_dict.items():
    #if key=='insignificant':
        #continue #skip "insignificant" key and its data
    durations = [tuple_item[4] for tuple_item in tuples_list]
    behavior= [tuple_item[0] for tuple_item in tuples_list]
    average_count = sum(durations)
    
    total_count_by_key[key] = average_count
    
     # Display the result for each key inside the loop
    print(f"Key: {key}, Average Count: {average_count}")

table_data = [(key, avg_count) for key, avg_count in total_count_by_key.items()]
headers = ["Key", "Average Count"]


table = tabulate(table_data, headers, tablefmt="pretty")
print(table)

##  Number of Bouts

In [ ]:
bout_by_key = {}

for key, tuples_list in bdbt_dict.items():
    #if key=='insignificant':
        #continue #skip "insignificant" key and its data
    bouts = [tuple_item[3] for tuple_item in tuples_list]
    sum_of_bouts = len(bouts)
    bout_by_key[key] = sum_of_bouts

table_data = [(key, sum_value) for key, sum_value in bout_by_key.items()]
headers = ["Key", "Total Bouts"]

table = tabulate(table_data, headers, tablefmt="pretty")
print(table)

# Create a dictionary for the CSV data
csv_data_bouts = {f'{fol3[0]} {fol4[1]}': bout_by_key}
# Plot the data
plt.figure(figsize=(10, 6))
plt.bar(bout_by_key.keys(), bout_by_key.values())
plt.xlabel("Key")
plt.ylabel("Total Bouts")
plt.title(f'Total Bouts by Key {fol3[0]} {fol4[1]}')
plt.xticks(rotation=45, ha='right')  # Set the x-axis ticks and labels
plt.tight_layout()
# Check if the user selected any files
if infiles:
    # Assuming infiles is a list of selected file paths, let's use the first one
    selected_file = infiles[0]

    # Extract the directory path from the selected file's full path
    directory_path = os.path.dirname(selected_file)
    # Save the plot as "(specific animals) Bout.png"
    filename_bout= f'{fol3[0]} {fol4[1]} Bouts.png'
    save_path= os.path.join(directory_path,filename_bout)
    plt.savefig(save_path)

    # Save the CSV data to a CSV file
    csv_filename_bouts = f'{fol3[0]} {fol4[1]} Bouts.csv'
    csv_save_path = os.path.join(directory_path, csv_filename_bouts)
    
    with open(csv_save_path, mode="w", newline="") as csv_file:
        csv_writer = csv.writer(csv_file)
        for dictionary_name, data_dict in csv_data_bouts.items():
            csv_writer.writerow([dictionary_name])  # Write the dictionary name as a header
            csv_writer.writerow(["Key", "Total Bouts"])
            for key, sum_value in data_dict.items():
                csv_writer.writerow([key, sum_value])
    print(f"CSV data saved in: {csv_save_path}")

# Check!!!

In [ ]:
print(csv_save_path)

In [ ]:
for file_path in infiles:  # if taking from Z Drive 
    folders = os.path.dirname(file_path).rsplit('\\', 1)
    print(folders)

In [ ]:
import pandas as pd

# Define the file paths for the two CSV files and the output CSV file
bout_csv_filename = os.path.join('/'.join(folders), csv_filename_bouts)
duration_csv_filename = os.path.join('/'.join(folders), csv_filename )

# Read the CSV files into DataFrames
bout_df = pd.read_csv(bout_csv_filename, skiprows=[0])
duration_df = pd.read_csv(duration_csv_filename, skiprows=[0])

print("Columns in bout_df:", bout_df.columns)
print("Columns in duration_df:", duration_df.columns)

In [ ]:
# Merge the two DataFrames on the shared key column (e.g., "Key")
merged_df = bout_df.merge(duration_df, on="Key", how="inner")

# Perform the multiplication: Total Bouts * Average Duration (seconds)
merged_df["Result(seconds)"] = merged_df["Total Bouts"] * merged_df["Average Duration (seconds)"]

# Display the results DataFrame within the Jupyter Notebook
display(merged_df)


In [ ]:
# Calculate the sum of the "Result" column in the DataFrame
total_result = (merged_df["Result(seconds)"].sum())/60

print(f"Sum of the 'Result' column: {total_result} minutes")


## Creating CSV

In [ ]:
rounded_dict = {}

for key, value_list in bdbt_dict.items():
    rounded_values = [
        (
            round(v[0]),     # Rounding and converting to integer
            v[1], v[2], v[3], 
            round(v[4], 3),
            round(v[5], 2),  
            round(v[6], 2)   
        )
        for v in value_list
    ]
    rounded_dict[key] = rounded_values

#print(rounded_dict)

bdbt_dict=rounded_dict
#print(bdbt_dict)

In [ ]:
from json import dumps
string_bdbt_dict=dumps(bdbt_dict)

In [ ]:
# Find the start index of the first '{' character
start_index = string_bdbt_dict.index('{')

# Find the end index of the last '}' character
end_index = string_bdbt_dict.rindex('}') + 1

# Extract the dictionary portion of the string
dict_str = string_bdbt_dict[start_index:end_index]

# Find the start indices of key-value pairs using '":'
key_value_end = []
for pos, char in enumerate(dict_str):
    if dict_str[pos:pos+3] == '": ':
        key_value_end.append(pos)
        
#print(key_value_end)      

# Initialize lists to store extracted keys and values
keys = []
values = []
value_ends=[]

# Iterate through the start indices of key-value pairs
for key_value_start in key_value_end:
    # Find the start index of the value by searching for '[[' after the key-value pair
    value_start = dict_str.find('[[', key_value_start) + 1
    
    # Find the end index of the value by searching for ']]' after the value_start
    value_end = dict_str.find(']]', value_start)+ 2
    value_ends.append(value_end)

    # Extract the value using the calculated start and end indices
    value = dict_str[value_start:value_end]
    values.append(value)
    # Extract the key using the portion of the string before the '":'
    if key_value_start == key_value_end[0]:
        key = dict_str[2:key_value_start].strip()
        keys.append(key)
    else:
        key = dict_str[value_ends[-2]+ 3:key_value_start].strip()
        keys.append(key)
        
#print(values)
#print(value_ends)
modified_values = {}  # Initialize an empty dictionary to store modified values

# Iterate through the keys and values
for key, value in zip(keys, values):
    modified_value = value.replace(" ", "").replace("],", "]").replace("]]","]")
    modified_values[key] = modified_value

In [ ]:
keys_to_write = ["Rear", "Investigate Type 1", "Investigate Type 2",
                 "Contra-Itch", "Investigate Type 3", "Contra-Body Groom",
                 "Face Groom Type 1", "Dive/Scrunch", "Head Groom", "Ipsi-Orient",
                 "Face Groom Type 2", "Ipsi-Body Groom", "Ipsi-Itch Type 1",
                 "Paw-Groom", "Locomotion", "Contra-Orient"]

In [ ]:
# Create a dictionary to hold cleaned and possibly split values
cleaned_values = {}

In [ ]:
# Iterate through the keys and values, cleaning and splitting if needed
for key in keys_to_write:
    value = modified_values[key].strip()
    
    if len(value) > 32500:
        # Find the last "]" before 32500 characters
        split_index = value.rfind("]", 0, 32500)
        
        if split_index != -1:
            # Split the value into two parts
            part1 = value[:split_index + 1]  # Include the last "]"
            part2 = value[split_index + 1:]
            
            cleaned_values[key] = part1  # Store the first part
            
            continue_key = f"{key} Continue"
            continue_count = 1
            
            while len(part2) > 32500:
                continue_count += 1
                split_index = part2.rfind("]", 0, 32500)  # Find the last "]" in the remaining portion
                part1 = part2[:split_index + 1]
                part2 = part2[split_index + 1:]
                cleaned_values[f"{key} Continue {continue_count}"] = part1
            cleaned_values[f"{key} Continue {continue_count + 1}"] = part2  # Store the remaining part
            
        else:
            # If "]" is not found, just store the value as it is
            cleaned_values[key] = value
    else:
        cleaned_values[key] = value


In [ ]:
# Format specific values in cleaned_values dictionary
formatted_values = {'"{}"'.format(key): value for key, value in cleaned_values.items()}

In [ ]:
# Initialize the list of behavior keys you want to include in mouse_data
behavior_keys = list(cleaned_values.keys())

In [ ]:
print(behavior_keys)

In [ ]:
# Specify the output path for the CSV file
output_path = r"Z:\KayCei\Practice 16.csv" #if on Yttri-Lab
output_path=r"Users/Shared/K/KayCei/Practice 16.csv"

data_dict = {}
session_numbers = {}

# Iterate through each file in infiles
for file_path in infiles: # if taking from Z Drive 
    folders = (os.path.dirname(file_path)).split('/')
    print(folders)
    # Extract the mouse name from the file path
    mouse_name = fol3[0]
    # Extract the genotype from the file path
    genotype = fol3[1]
    # Extract the date from the file path
    date = fol4[0]
    environment= fol4[1]
#for file_path in infiles:   #if taking files from D drive
    #folders = (os.path.dirname(file_path)).split('/')
    #print(folders)
    # Extract the mouse name from the file path
    #mouse_name = folders[6].split('-')[0]
    # Extract the genotype from the file path
    #genotype = folders[6].split('-')[1]
    # Extract the date from the file path
    #date = folders[-1].split('-')[0]    
# Determine the session number for this mouse and stim behavior
    key = (mouse_name, environment)
    if key not in session_numbers:
        session_numbers[key] = 1
    else:
        session_numbers[key] += 1
    session_number = session_numbers[key]

   # Initialize the mouse_data dictionary
    mouse_data = {
        "Mouse": mouse_name,
        "Genotype": genotype,
        "Date": date,
        "Environment": environment,
        "Session Number": session_number,
    }

    for behavior_key in behavior_keys:
        cleaned_value = cleaned_values.get(behavior_key, 0)
        mouse_data[behavior_key] = cleaned_value

        if not behavior_key.endswith("Continue"):
            bout_count = bout_by_key.get(behavior_key, 0)
            average_duration = average_duration_by_key.get(behavior_key, 0)

            mouse_data[f"{behavior_key} Total Bout Count"] = bout_count
            mouse_data[f"{behavior_key} Average Duration(s)"] = average_duration

            # Check if the behavior_key ends with "Continue"
            if behavior_key.endswith("Continue"):
                # Remove the "Total bout" and "Average duration" columns
                del mouse_data[f"{behavior_key} Total Bout Count"]
                del mouse_data[f"{behavior_key} Average Duration(s)"]

    # Add mouse_data to data_dict
    data_dict[mouse_name] = mouse_data

In [ ]:
import re

# Define a pattern to match "Continue" followed by one or more digits
pattern = r"Continue \d+ (Total Bout Count|Average Duration\(s\))"

# Create a list of keys to remove based on the pattern
keys_to_remove = [key for key in mouse_data.keys() if re.search(pattern, key)]

# Remove the keys and their associated values from the dictionary
for key in keys_to_remove:
    del mouse_data[key]

# Print the updated dictionary
#print(mouse_data)

In [ ]:
#print(keys_to_remove)

In [ ]:
# Using a list comprehension
#keys = [key for key in mouse_data.keys()]
#print(keys)

In [ ]:
# Define the headers for the CSV file
headers = list(mouse_data.keys())

In [ ]:
print(headers)

In [ ]:
if os.path.exists(output_path):
    # Keep track of existing entries in the CSV to avoid duplicates
    existing_entries = set()
    entry_key2_session_numbers = {}
    sorted_entries = []

    # Read existing data from the CSV and add it to the set
    with open(output_path, 'r') as csvfile:
        reader = csv.DictReader(csvfile, delimiter=';')
        for row in reader:
            entry_key = (row["Mouse"], row["Date"], row["Environment"])
            entry_key2 = (row["Mouse"], row["Date"], row["Environment"], row["Genotype"])
            existing_entries.add(entry_key)
            entry_key2_session_numbers[entry_key2] = int(row["Session Number"])
            sorted_entries.append(row)
        
else:
    existing_entries = set()
    entry_key2_session_numbers = {}
    
# Ensure the output directory exists before writing the CSV file
output_dir = os.path.dirname(output_path)
os.makedirs(output_dir, exist_ok=True)

# Write the data from data_dict to the CSV file (in append mode 'a')
with open(output_path, 'a', newline='') as csvfile:
    writer = csv.DictWriter(csvfile, fieldnames=headers, delimiter=';')

    # Check if the file is empty, if so, write the header
    if csvfile.tell() == 0:
        writer.writeheader()

for mouse_data in data_dict.values():
    entry_key = (mouse_data["Mouse"], mouse_data["Date"], mouse_data["Environment"])
    entry_key2 = (mouse_data["Mouse"], mouse_data["Date"], mouse_data["Environment"])

    # Check if the entry already exists in the CSV
    if entry_key not in existing_entries:
        session_number = entry_key2_session_numbers.get(entry_key2, 0) + 1
        existing_entries.add(entry_key)
        entry_key2_session_numbers[entry_key2] = session_number

        for formatted_key in behavior_keys:
            if formatted_key in formatted_values:
                mouse_data[formatted_key] = formatted_values[formatted_key]

        # Write the complete mouse data to the CSV
        sorted_entries.append(mouse_data)  # Add the new entry

        print("Entry added Successfully:", entry_key)
    else:
        print("Entry already exists in the CSV:", entry_key)

In [ ]:
# Add unique keys from unique_entry_keys to fieldnames
fieldnames=headers

In [ ]:
# Convert the fieldnames list to a set
fieldnames_set = set(headers)  # Replace 'headers' with your actual fieldnames list

# Extract the keys from the dictionaries within sorted_entries and create a set
entry_keys_set = set()
for entry in sorted_entries:
    entry_keys_set.update(entry.keys())

# Find the unique keys in fieldnames_set and entry_keys_set
unique_fieldnames = fieldnames_set.difference(entry_keys_set)
unique_entry_keys = entry_keys_set.difference(fieldnames_set)

# Print the unique keys
print("Unique keys in fieldnames_set:", unique_fieldnames)
print("Unique keys in entry_keys_set:", unique_entry_keys)


In [ ]:
for unique_entry_key in unique_entry_keys:
    # Check if unique_entry_key exists (not None and not an empty string)
    if unique_entry_key:
        # Extract the prefix by splitting at the first space followed by a digit
        prefix_to_match = re.split(r'\s+\d', unique_entry_key)[0]

        # Find the position to insert unique_entry_key
        insert_position = None

        for i in range(len(fieldnames) - 1, -1, -1):
            if fieldnames[i].startswith(prefix_to_match):
                insert_position = i + 1
                break

        # If the prefix is not found, append the key to the end
        if insert_position is None:
            insert_position = len(fieldnames)

        # Insert the unique_entry_key at the specified position
        fieldnames.insert(insert_position, unique_entry_key)
    else:
        # Skip to the next iteration (next unique_entry_key)
        continue

In [ ]:
print(fieldnames)

In [ ]:
# Sort the entries based on date within each group of entries that share the same entry_key2
sorted_entries.sort(key=lambda x: (x["Mouse"], x["Date"], x["Environment"]))

# Update the session numbers based on the sorted order
entry_key2_counter = {}  # To track the session numbers within each entry_key2 group

for entry in sorted_entries:
    entry_key2 = (entry["Mouse"], entry["Date"], entry["Environment"], entry["Genotype"])
    
    if entry_key2 not in entry_key2_counter:
        entry_key2_counter[entry_key2] = 1
    else:
        entry_key2_counter[entry_key2] += 1
    
    entry["Session Number"] = entry_key2_counter[entry_key2]

# Sort the entries based on date within each group of entries that share the same entry_key2
sorted_entries.sort(key=lambda x: (x["Mouse"], x["Date"], x["Environment"], x["Genotype"]))

# Rewrite the CSV file with sorted and formatted entries, including any missing fields
with open(output_path, 'w', newline='') as csvfile:
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames, delimiter=';')
    writer.writeheader()
    writer.writerows(sorted_entries)
